In [ ]:
# This notebook is by Anastasia Ruzmaikina. It was submitted to Kaggle Competition on LLM-Detect AI Generated Text

In recent years, large language models (LLMs) have become increasingly sophisticated, capable of generating text that is difficult to distinguish from human-written text. In this competition, we hope to foster open research and transparency on AI detection techniques applicable in the real world.

This competition challenges participants to develop a machine learning model that can accurately detect whether an essay was written by a student or an LLM. The competition dataset comprises a mix of student-written essays and essays generated by a variety of LLMs.

Can you help build a model to identify which essay was written by middle and high school students, and which was written using a large language model? With the spread of LLMs, many people fear they will replace or alter work that would usually be done by humans. Educators are especially concerned about their impact on students’ skill development, though many remain optimistic that LLMs will ultimately be a useful tool to help students improve their writing skills.

This notebook uses the dataset of AI written essays generated by the author using various LLMs and compares it with the dataset of human written essays provided in the competition. The model used to distinguish between human-written and AI-written essays is Llama-2-7b, and the best result on the competition test dataset is 85.1% accuracy.


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

This notebook is adapted from the Notebook by Yuichi Tateno:  LLM detect AI comp Mistral-7B
https://www.kaggle.com/code/hotchpotch/train-llm-detect-ai-comp-mistral-7b

The only major changes I have made were: use Llama2-7b-hf model from Kaggle datasets and to use my own generated.csv dataset for AI generated essays. In addition some small changes were made to the notebook.

In addition I have included the 'daight-pip' dataset from the Notebook by Min-Hsien Weng: TPU train Mistral 7b|Llama 2 detection
https://www.kaggle.com/code/minhsienweng/tpu-train-mistral-7b-llama-2-detection

In [ ]:
# Install package for inferences
!pip install -qq --no-deps /kaggle/input/daigt-pip/peft-0.6.0-py3-none-any.whl
!pip install -qq --no-deps /kaggle/input/daigt-pip/transformers-4.35.0-py3-none-any.whl
!pip install -qq --no-deps /kaggle/input/daigt-pip/tokenizers-0.14.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install -qq --no-deps /kaggle/input/daigt-pip/optimum-1.14.0-py3-none-any.whl

In [ ]:
!pip install -qq --no-deps /kaggle/input/llm-detect-pip/accelerate-0.24.1-py3-none-any.whl
!pip install -qq --no-deps /kaggle/input/llm-detect-pip/bitsandbytes-0.41.1-py3-none-any.whl


In [ ]:
from __future__ import annotations

TARGET_MODEL = '/kaggle/input/llama2-7b-hf/Llama2-7b-hf'#"mistralai/Mistral-7B-v0.1"

DEBUG = False

In [ ]:

# ====================================================
# Directory settings
# ====================================================
from pathlib import Path

OUTPUT_DIR = Path("./")
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

INPUT_DIR = Path("../input/")

In [ ]:
import pandas as pd
train_df = pd.read_csv(INPUT_DIR / "llm-detect-ai-generated-text/train_essays.csv", sep=',')
test_df = pd.read_csv(INPUT_DIR / "llm-detect-ai-generated-text/test_essays.csv", sep=',')
#external_df = pd.read_csv('/kaggle/input/generated2/generated.csv', sep=',')
external_df1 = pd.read_csv('/kaggle/input/generated-classes/generatedchat.csv', sep=',')
external_df1 = external_df1.sample(600, random_state=42)
external_df2 = pd.read_csv('/kaggle/input/generated-classes/generatedsumm.csv', sep=',')
external_df2 = external_df2.sample(600, random_state=42)
external_df3 = pd.read_csv('/kaggle/input/generated-llama/generatedLlama.csv')
external_df4 = pd.read_csv('/kaggle/input/generated-llama/generatedLlama.csv')
external_df5 = pd.read_csv('/kaggle/input/generated-llama/generatedLlama.csv')
external_df6 = pd.read_csv('/kaggle/input/generated-gpt/SummaryGpt.csv')
#external_df7 = pd.read_csv('/kaggle/input/generated-gpt/SummaryGpt.csv')
#external_df8 = pd.read_csv('/kaggle/input/generated-gpt/SummaryGpt.csv')
external_df9 = pd.read_csv('/kaggle/input/generated-gpt3-5/SummaryGpt3.5.csv')
external_df6 = external_df6.drop('Unnamed: 0', axis=1)
#external_df10 = pd.read_csv('/kaggle/input/generated-rnn/SummaryRNN.csv')
#external_df10['text'] = external_df10['text'].apply(lambda x: x[2:-1])
#external_df7 = external_df7.drop('Unnamed: 0', axis=1)
#external_df8 = external_df8.drop('Unnamed: 0', axis=1)
external_df11 = pd.read_csv("/kaggle/input/generated-llama2/SummaryLlama5.csv")
#external_df = pd.concat([external_df1,external_df2, external_df3, external_df4, external_df5], ignore_index = True) 
#external_df = pd.concat([external_df1,external_df2, external_df3, external_df4, external_df5, external_df9, external_df11], ignore_index = True) 
external_df = pd.concat([external_df1,external_df2, external_df3, external_df4, external_df5], ignore_index = True) 
#external_df = pd.concat([external_df1,external_df2, external_df3, external_df4, external_df5, external_df6, external_df7, external_df8], ignore_index = True) 
#external_df = pd.concat([external_df1,external_df2, external_df3, external_df4, external_df5, external_df6, external_df9], ignore_index = True) 
#external_df = pd.concat([external_df1,external_df2, external_df3, external_df4, external_df5, external_df6, external_df9, external_df10], ignore_index = True) 

train_prompts_df = pd.read_csv(INPUT_DIR / "llm-detect-ai-generated-text/train_prompts.csv", sep=',')

# show shape
print(f'train_df.shape: {train_df.shape}')
print(f'test_df.shape: {test_df.shape}')
print(f'external_df.shape: {external_df.shape}')
print(f'train_prompts_df.shape: {train_prompts_df.shape}')

In [ ]:
!pip install -q -U peft --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U accelerate --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U bitsandbytes --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U transformers --no-index --find-links ../input/llm-detect-pip/
# Install package for inferences
!pip install -qq --no-deps /kaggle/input/daigt-pip/peft-0.6.0-py3-none-any.whl
!pip install -qq --no-deps /kaggle/input/daigt-pip/transformers-4.35.0-py3-none-any.whl
!pip install -qq --no-deps /kaggle/input/daigt-pip/tokenizers-0.14.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install -qq --no-deps /kaggle/input/daigt-pip/optimum-1.14.0-py3-none-any.whl
!pip install -qq --no-deps /kaggle/input/llm-detect-pip/accelerate-0.24.1-py3-none-any.whl
!pip install -qq --no-deps /kaggle/input/llm-detect-pip/bitsandbytes-0.41.1-py3-none-any.whl


In [ ]:
# rename column generated to label
train_df = train_df.rename(columns={'generated': 'label'})
test_df = test_df.rename(columns={'generated': 'label'})
external_df = external_df.rename(columns={'generated': 'label'})


In [ ]:
#external_df6 = external_df6.drop('Unnamed: 0', axis=1)
#external_df10.text[0]


In [ ]:
train_df.label.value_counts()

In [ ]:
train_df.head(3)

In [ ]:
test_df.head(3)

In [ ]:
external_df.head(3)

In [ ]:
external_df.label.value_counts()

In [ ]:
train_df = pd.concat([train_df, external_df])
train_df = train_df.drop(['id','prompt_id'], axis=1)
train_df.reset_index(inplace=True, drop=True)
print(f"Train dataframe has shape: {train_df.shape}")
train_df.head()

In [ ]:
#train_df = train_df.sample(frac=0.1)
train_df.shape

In [ ]:
train_df.value_counts("label")

In [ ]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
X = train_df.loc[:, train_df.columns != "label"]
y = train_df.loc[:, train_df.columns == "label"]

for i, (train_index, valid_index) in enumerate(skf.split(X, y)):
    train_df.loc[valid_index, "fold"] = i
    
print(train_df.groupby("fold")["label"].value_counts())
train_df.head()

In [ ]:
# fold0 as valid
valid_df = train_df[train_df["fold"] == 0]
train_df = train_df[train_df["fold"] != 0]
print(train_df.shape)
print(valid_df.shape)

In [ ]:
# load model with 4bit bnb

from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType # type: ignore
from transformers import BitsAndBytesConfig
import torch

peft_config = LoraConfig(
    r=4,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    target_modules=[
        "q_proj",
        "v_proj"
    ],
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
from transformers import AutoTokenizer, LlamaForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained(TARGET_MODEL, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
base_model = LlamaForSequenceClassification.from_pretrained(
    TARGET_MODEL,
    num_labels=2,
    quantization_config=bnb_config,
    device_map={"":0}
)
base_model.config.pretraining_tp = 1 # 1 is 7b
base_model.config.pad_token_id = tokenizer.pad_token_id

In [ ]:
model = get_peft_model(base_model, peft_config)


In [ ]:
model.print_trainable_parameters()


In [ ]:
#remove this for actual calculation  here we take a smaller sample of the dataframe to speed up 
train_df = train_df.sample(2000, random_state=42)  #2400
df2 = pd.DataFrame({'text': ['aaaaa bbbbb ccccc.', 'bbbbbb cccccc dddddd.', 'cccc dddd eeee.', 'dd ee ff.'], 'label': [1,1,1,1], 'fold': [1.0,2.0,3.0,4.0]} )
for i in range(50):
    train_df = pd.concat([train_df,df2], ignore_index = True) 
print(train_df.tail())
print(train_df.label.value_counts(), valid_df.label.value_counts())

In [ ]:
#remove this for actual calculation
#df_sampled = df.sample(frac=0.4)
#df_remaining = df.loc[~df.index.isin(df_sampled.index)]

valid_df1 = valid_df.sample(frac = 0.4, random_state=42)
valid_df = valid_df.loc[~valid_df.index.isin(valid_df1.index)]
print(train_df.shape)
print(valid_df.shape)
print(valid_df1.shape)

In [ ]:
#from transformers import pipeline
#generator = pipeline('text-generation', model='gpt2')
#generator("Hello world, continue... ")


In [ ]:
train_df

In [ ]:
# datasets
from datasets import Dataset

# from pandas
train_ds = Dataset.from_pandas(train_df)
valid_ds = Dataset.from_pandas(valid_df)
test_ds = Dataset.from_pandas(test_df)
valid_ds1 = Dataset.from_pandas(valid_df1)

In [ ]:
def preprocess_function(examples, max_length=512):
    return tokenizer(examples["text"], truncation=True, max_length=max_length, padding=True)

In [ ]:
train_tokenized_ds = train_ds.map(preprocess_function, batched=True)
valid_tokenized_ds = valid_ds.map(preprocess_function, batched=True)
test_tokenized_ds = test_ds.map(preprocess_function, batched=True)
valid_tokenized_ds1 = valid_ds1.map(preprocess_function, batched=True)

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="longest")

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, roc_auc_score
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    accuracy_val = accuracy_score(labels, predictions)
    roc_auc_val = roc_auc_score(labels, predictions)
    
    return {
        "accuracy": accuracy_val,
        "roc_auc": roc_auc_val,
    }

In [ ]:
from transformers import TrainingArguments, Trainer

steps = 5 if DEBUG else 20

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    learning_rate=5e-4,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=16,
    max_grad_norm=0.3,
    optim='paged_adamw_32bit',
    lr_scheduler_type="cosine",
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
    push_to_hub=False,
    warmup_steps=steps,
    eval_steps=steps,
    logging_steps=steps,
    report_to='none' # if DEBUG else 'wandb',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized_ds,
    eval_dataset=valid_tokenized_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
#from shutil import rmtree

#trainer.save_model(output_dir=str(OUTPUT_DIR))

#for path in Path(training_args.output_dir).glob("checkpoint-*"):
    #if path.is_dir():
        #rmtree(path)

In [ ]:
#this is to check if the predictions work on a known dataframe not previously seen by the model
preds = trainer.predict(valid_tokenized_ds1)
logits = preds.predictions

In [ ]:
# from scipy.special import expit as sigmoid
import numpy as np
def sigmoid(x):
    return 1 / (1 + np.exp(-x))  
probs = sigmoid(logits[:, 1])
probs.shape, probs[0:5]


In [ ]:
#this is to check if the predictions work on a known dataframe not previosly seen by the model
valid_df1['preds'] = probs
valid_df1

In [ ]:
count = valid_df1[(valid_df1['preds'] >= 0.5 ) & (valid_df1['label'] == 1)].shape[0] + valid_df1[(valid_df1['preds'] < 0.5 ) & (valid_df1['label'] == 0)].shape[0]
print(count, count/valid_df1.shape[0])

In [ ]:
#this is for the predictions on the test set
preds = trainer.predict(test_tokenized_ds)#.predictions.astype(float)
logits = preds.predictions
#print(preds)
#preds = np.clip(preds, 0, 1)


In [ ]:
# from scipy.special import expit as sigmoid
import numpy as np
def sigmoid(x):
    #if x > -100:
        return 1 / (1 + np.exp(-x))  
   # else:
       # return 0
probs = sigmoid(logits[:, 1])
probs.shape, probs[0:5]


In [ ]:
print(sigmoid(10))
print(sigmoid(-10))
print(sigmoid(-1001))

In [ ]:
#this is to provide the sample submission
sub = pd.DataFrame()
test_df = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')
#sub['id'] = valid_df['id']
sub['id'] = test_df['id']
import math
sub['generated'] = probs
sub['generated'] = sub['generated'].round(1)
sub.to_csv('/kaggle/working/submission.csv', index=False)
sub.head()

In [ ]:
dfs = pd.read_csv('/kaggle/working/submission.csv')
dfs

In [ ]:
#del trainer, model, base_model

In [ ]:
# cuda cache clear
#import torch
#torch.cuda.empty_cache()